<h1> House prices prediciton </h1>

In this project, I'm going to analyze the data, clean the data and create e regression model to try to predict the house prices based on the several parameters given.

In [21]:
#import necessary libraries for data analysis, data visualization ad machine learning.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [22]:
correlation_factor = 0.1